In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CG PySpark SQL analytics").getOrCreate()

22/10/28 17:14:46 WARN Utils: Your hostname, hadoop resolves to a loopback address: 127.0.1.1; using 192.168.242.137 instead (on interface ens33)
22/10/28 17:14:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/28 17:14:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/28 17:14:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/28 17:14:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = spark.read.json("/usr/local/spark/examples/src/main/resources/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [4]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
df.select(df["name"],df['age'] + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [7]:
df.filter(df['age'] > 20).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [8]:
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [9]:
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [10]:
# Register the DataFrame as a global temporary view 

df.createGlobalTempView("people")
spark.sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [11]:
#Global temporary View is to make cross-session
spark.newSession().sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [15]:
from pyspark.sql import SparkSession

In [16]:
from pyspark.sql import Row 

sc = spark.sparkContext

In [17]:
lines = sc.textFile("/usr/local/spark/examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

In [19]:
#Infer the schema , register the Dataframe as a table

schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

In [20]:
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 20")

In [21]:
# Add the results of SQL queries with Dataframe objects
teenNames = teenagers.rdd.map(lambda p: "Name:" + p.name).collect()
for name in teenNames:
    print(name)

Name:Justin
